In [1]:
import numpy as np
import pandas as pd

In [3]:
df=pd.read_csv("diabetes.csv")

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
X=df.iloc[:,:-1].values # convert to numpy array
y=df.iloc[:,-1].values

In [6]:
X

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [7]:
X.shape

(768, 8)

In [8]:
y.shape

(768,)

In [9]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()


In [10]:
X=scaler.fit_transform(X)

In [11]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.20, random_state=42)

In [14]:
import tensorflow as tf


In [15]:
LAYERS=[
    tf.keras.layers.Dense(32,activation='relu', input_dim=8),
    tf.keras.layers.Dense(1,activation='sigmoid')
]
model=tf.keras.models.Sequential(LAYERS)

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
LOSS_FUNCTION='binary_crossentropy'
OPTIMIZER='Adam'
METRICS=['accuracy']
model.compile(loss=LOSS_FUNCTION,optimizer=OPTIMIZER,metrics=METRICS)

In [18]:
model.fit(X_train, y_train,batch_size=32,epochs=10, validation_data=(X_test,y_test))

Epoch 1/10
20/20 [==============================] - 6s 13ms/step - loss: 0.6363 - accuracy: 0.6873 - val_loss: 0.6508 - val_accuracy: 0.6753
Epoch 2/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5846 - accuracy: 0.7182 - val_loss: 0.6086 - val_accuracy: 0.7273
Epoch 3/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5516 - accuracy: 0.7410 - val_loss: 0.5813 - val_accuracy: 0.7208
Epoch 4/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5303 - accuracy: 0.7622 - val_loss: 0.5619 - val_accuracy: 0.7468
Epoch 5/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5149 - accuracy: 0.7687 - val_loss: 0.5515 - val_accuracy: 0.7597
Epoch 6/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5042 - accuracy: 0.7606 - val_loss: 0.5417 - val_accuracy: 0.7532
Epoch 7/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4945 - accuracy: 0.7622 - val_loss: 0.5338 - val_accuracy: 0.7662
Epoch 8/10
20/20 [=

In [ ]:
# HOW TO SELECT BEST OPTIMIZER
# HOW TO SELECT NUMBER OF NODES OR NEURONS IN LAYER
# HOW TO SELECT NUMBER OF LAYERS
# ALL IN ONE MODEL

In [20]:
!pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.9 MB/s eta 0:00:00


In [21]:
import keras_tuner as kt

In [ ]:
# HOW TO SELECT BEST OPTIMIZER

In [22]:
def build_model(hp):
  LAYERS=[
    tf.keras.layers.Dense(32,activation='relu', input_dim=8),
    tf.keras.layers.Dense(1,activation='sigmoid')]
  model=tf.keras.models.Sequential(LAYERS)
  LOSS_FUNCTION='binary_crossentropy'
  OPTIMIZER=hp.Choice('optimizer', values=['adam','sgd','rmsprop','adadelta'])
  METRICS=['accuracy']
  model.compile(loss=LOSS_FUNCTION,optimizer=OPTIMIZER,metrics=METRICS)
  return model


In [24]:
tuner = kt.RandomSearch(build_model,
                      objective='val_accuracy',#validation accuracy
                      max_trials=5)

In [26]:
tuner.search(X_train,y_train,epochs=5, validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.6623376607894897

Best val_accuracy So Far: 0.6818181872367859
Total elapsed time: 00h 00m 08s


In [30]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [31]:
model=tuner.get_best_models(num_models=1)[0]

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
model.fit(X_train,y_train,epochs=10,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/10
20/20 [==============================] - 1s 12ms/step - loss: 0.5327 - accuracy: 0.7215 - val_loss: 0.5859 - val_accuracy: 0.6883
Epoch 8/10
20/20 [==============================] - 0s 6ms/step - loss: 0.5151 - accuracy: 0.7296 - val_loss: 0.5726 - val_accuracy: 0.6818
Epoch 9/10
20/20 [==============================] - 0s 6ms/step - loss: 0.5013 - accuracy: 0.7492 - val_loss: 0.5618 - val_accuracy: 0.6818
Epoch 10/10
20/20 [==============================] - 0s 6ms/step - loss: 0.4898 - accuracy: 0.7557 - val_loss: 0.5541 - val_accuracy: 0.6948


In [ ]:
# HOW TO SELECT No.of nodes/neurons

In [34]:
def build_model(hp):
  units=hp.Int('units',min_value=8, max_value=128, step=8)
  LAYERS=[
    tf.keras.layers.Dense(units=units,activation='relu', input_dim=8),
    tf.keras.layers.Dense(1,activation='sigmoid')]
  model=tf.keras.models.Sequential(LAYERS)
  LOSS_FUNCTION='binary_crossentropy'
  OPTIMIZER='rmsprop'
  METRICS=['accuracy']
  model.compile(loss=LOSS_FUNCTION,optimizer=OPTIMIZER,metrics=METRICS)
  return model

In [37]:
tuner = kt.RandomSearch(build_model,
                      objective='val_accuracy',#validation accuracy
                      max_trials=5,directory='my_dir')

In [38]:
tuner.search(X_train,y_train,epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 09s


In [40]:
tuner.get_best_hyperparameters()[0].values

{'units': 80}

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# HOW TO SELECT NUMBER OF LAYERS

In [42]:
import tensorflow as tf
import keras
from keras.layers import Dense
from keras.models import Sequential

In [43]:
def build_model(hp):
  model=Sequential()
  model.add(Dense(80,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(80,activation='relu',input_dim=8))

  model.add(Dense(1,activation='sigmoid'))

  LOSS_FUNCTION='binary_crossentropy'
  OPTIMIZER='rmsprop'
  METRICS=['accuracy']
  model.compile(loss=LOSS_FUNCTION,optimizer=OPTIMIZER,metrics=METRICS)

  return model

In [45]:
tuner = kt.RandomSearch(build_model,
                      objective='val_accuracy',#validation accuracy
                      max_trials=5,directory='my_dir1')

In [46]:
tuner.search(X_train,y_train,epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7662337422370911

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 15s


In [47]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4}

In [48]:
model=tuner.get_best_models(num_models=1)[0]

In [49]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                720       
                                                                 
 dense_1 (Dense)             (None, 80)                6480      
                                                                 
 dense_2 (Dense)             (None, 80)                6480      
                                                                 
 dense_3 (Dense)             (None, 80)                6480      
                                                                 
 dense_4 (Dense)             (None, 80)                6480      
                                                                 
 dense_5 (Dense)             (None, 1)                 81        
                                                                 
Total params: 26721 (104.38 KB)
Trainable params: 26721 

## ALL IN ONE

In [50]:
def build_model(hp):

  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers',min_value= 1,max_value=10)):
    if counter == 0:
      model.add(Dense(hp.Int('units' + str(i),min_value= 8,max_value=128, step=8),
                      activation=hp.Choice('activation' + str(i), values=['relu', 'tanh']),
                      input_dim=8))


    else:
      model.add(Dense(hp.Int('units' + str(i),min_value= 8,max_value=128, step=8),
                      activation=hp.Choice('activation' + str(i), values=['relu', 'tanh'])))

    counter += 1


  model.add(Dense(1, activation='sigmoid'))


  LOSS_FUNCTION = "binary_crossentropy"
  OPTIMIZER = hp.Choice('optimizer', values = ['adam', 'sgd', 'rmsprop', 'adadelta'])
  METRICS = ["accuracy"]

  model.compile(loss=LOSS_FUNCTION,
                optimizer=OPTIMIZER,
                metrics=METRICS)

  return model


In [51]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 5,
                        directory='mydir2')

In [52]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7727272510528564

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 25s


In [53]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 8,
 'units0': 104,
 'activation0': 'tanh',
 'optimizer': 'rmsprop',
 'units1': 32,
 'activation1': 'tanh',
 'units2': 56,
 'activation2': 'relu',
 'units3': 32,
 'activation3': 'tanh',
 'units4': 48,
 'activation4': 'relu',
 'units5': 48,
 'activation5': 'relu',
 'units6': 8,
 'activation6': 'relu',
 'units7': 8,
 'activation7': 'relu'}

In [54]:
model = tuner.get_best_models(num_models=1)[0]

In [55]:
model.fit(X_train, y_train, epochs=10, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/10
20/20 [==============================] - 3s 25ms/step - loss: 0.4489 - accuracy: 0.7932 - val_loss: 0.5270 - val_accuracy: 0.7468
Epoch 8/10
20/20 [==============================] - 0s 11ms/step - loss: 0.4427 - accuracy: 0.7850 - val_loss: 0.5083 - val_accuracy: 0.7792
Epoch 9/10
20/20 [==============================] - 0s 10ms/step - loss: 0.4419 - accuracy: 0.7834 - val_loss: 0.5136 - val_accuracy: 0.7597
Epoch 10/10
20/20 [==============================] - 0s 10ms/step - loss: 0.4372 - accuracy: 0.7948 - val_loss: 0.5001 - val_accuracy: 0.7727
